In [ ]:
import logging
import os
from contextlib import nullcontext
from pathlib import Path

import joblib
import matplotlib.pyplot as plt
import mlflow
import mlflow.sklearn
import numpy as np
import pandas as pd
import xgboost as xgb
import yaml
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    root_mean_squared_error,
)
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVR

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

MLFLOW_TRACKING_URI = 'http://localhost:5555'  # Optional: e.g., 'http://localhost:5555'